<a href="https://colab.research.google.com/github/dannygallagher/CIS519-Project/blob/main/Baselines/Baseline_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import gzip
import json
import json
import gzip
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.feature_extraction.text import TfidfTransformer
def get_metrics(true_labels, pred_labels):
    f1 = f1_score(true_labels, pred_labels, average='macro')
    acc = accuracy_score(true_labels, pred_labels)
    rec = recall_score(true_labels, pred_labels, average='macro')
    prec = precision_score(true_labels, pred_labels, average='macro')
    return f1, acc, rec, prec

def fast_parse(path):
  # path = path + ".json.gz"
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  all_reviews = []
  g = open(path, 'r')
  for line in g:
    d = json.loads(line)
    all_reviews.append(d)
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
    counter = counter + 1
  return all_reviews

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
total_results = pd.DataFrame(columns=col_list)

All 5 Cat

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/RandomForest/results_All_5Cat.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')

depth = [int(x) for x in np.linspace(10, 40, num = 5)]
# estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 5)]
grid = {'max_depth': depth}

rf = RandomForestClassifier(verbose = 1, n_jobs=-1)

GS_object = GridSearchCV(estimator = rf, param_grid = grid, verbose = 1)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['overall']
y_test = df_test['overall']
y_exp = df_exp['overall']


GS_object.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   37.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   36.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Pa

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              rand

In [ ]:
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = GS_object.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["5Cat", "Random Forest", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = GS_object.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["5Cat", "Random Forest", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = GS_object.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["5Cat", "Random Forest", "All", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    3.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    6.6s finished


65.87%


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


42.84%


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.6s


42.60%


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.4s finished


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Random Forest,All,All-Train,0.658685,0.661813,0.661848,0.708839
0,5Cat,Random Forest,All,All-Test,0.428357,0.623331,0.484019,0.406110
0,5Cat,Random Forest,All,Elec-Test,0.425952,0.470233,0.454368,0.444021


Elec 5 Cat

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/RandomForest/results_Elec_5Cat.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')

depth = [int(x) for x in np.linspace(10, 40, num = 5)]
# estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 5)]
grid = {'max_depth': depth}

rf = RandomForestClassifier(verbose = 1, n_jobs=-1)

GS_object = GridSearchCV(estimator = rf, param_grid = grid, verbose = 1)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['overall']
y_test = df_test['overall']
y_exp = df_exp['overall']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)

from sklearn.naive_bayes import MultinomialNB

GS_object.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   40.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   40.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Pa

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              rand

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = GS_object.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["5Cat", "Random Forest", "Elec", "Elec-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = GS_object.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["5Cat", "Random Forest", "Elec", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = GS_object.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["5Cat", "Random Forest", "Elec", "ALL-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    3.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    7.4s finished


69.45%


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.7s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


47.00%


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s


38.60%


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.2s finished


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Random Forest,Elec,Elec-Train,0.694476,0.695094,0.695243,0.753679
0,5Cat,Random Forest,Elec,Elec-Test,0.469977,0.502783,0.487694,0.489218
0,5Cat,Random Forest,Elec,ALL-Test,0.386031,0.622710,0.425438,0.372395


ALL Binary

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/RandomForest/results_All_bin.csv'
df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train_binaryposneg.json')
df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
df_exp = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
df_train = pd.DataFrame().from_dict(df_train)
df_test = pd.DataFrame().from_dict(df_test)
df_exp = pd.DataFrame().from_dict(df_exp)

depth = [int(x) for x in np.linspace(10, 40, num = 5)]
# estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 5)]
grid = {'max_depth': depth}

rf = RandomForestClassifier(verbose = 1, n_jobs=-1)

GS_object = GridSearchCV(estimator = rf, param_grid = grid, verbose = 1)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['pos_neg']
y_test = df_test['pos_neg']
y_exp = df_exp['pos_neg']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)


GS_object.fit(X_train, y_train)

On line0, 0.0 % done
On line5000, 0.002960002083841467 % done
On line10000, 0.005920004167682934 % done
On line15000, 0.008880006251524401 % done
On line20000, 0.011840008335365868 % done
On line25000, 0.014800010419207334 % done
On line30000, 0.017760012503048803 % done
On line35000, 0.02072001458689027 % done
On line40000, 0.023680016670731736 % done
On line45000, 0.026640018754573204 % done
On line50000, 0.02960002083841467 % done
On line55000, 0.03256002292225614 % done
On line60000, 0.035520025006097605 % done
On line65000, 0.03848002708993907 % done
On line70000, 0.04144002917378054 % done
On line75000, 0.04440003125762201 % done
On line80000, 0.04736003334146347 % done
On line85000, 0.050320035425304936 % done
On line90000, 0.05328003750914641 % done
On line95000, 0.05624003959298787 % done
On line100000, 0.05920004167682934 % done
On line105000, 0.06216004376067081 % done
On line110000, 0.06512004584451228 % done
On line115000, 0.06808004792835375 % done
On line120000, 0.071040

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Pa

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              rand

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = GS_object.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Binary", "Random Forest", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = GS_object.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Binary", "Random Forest", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = GS_object.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["Binary", "Random Forest", "All", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    4.9s finished


87.69%


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


74.39%


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s


82.34%


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Binary,Random Forest,All,All-Train,0.876949,0.877206,0.877183,0.880323
0,Binary,Random Forest,All,All-Test,0.743919,0.879429,0.848142,0.701758
0,Binary,Random Forest,All,Elec-Test,0.823400,0.827400,0.836942,0.820836


Elec Bin

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/RandomForest/results_Elec_Bin.csv'
df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train_binaryposneg.json')
df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
df_exp = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
df_train = pd.DataFrame().from_dict(df_train)
df_test = pd.DataFrame().from_dict(df_test)
df_exp = pd.DataFrame().from_dict(df_exp)

depth = [int(x) for x in np.linspace(10, 40, num = 5)]
# estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 5)]
grid = {'max_depth': depth}

rf = RandomForestClassifier(verbose = 1, n_jobs=-1)

GS_object = GridSearchCV(estimator = rf, param_grid = grid, verbose = 1)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['pos_neg']
y_test = df_test['pos_neg']
y_exp = df_exp['pos_neg']


GS_object.fit(X_train, y_train)

On line0, 0.0 % done
On line5000, 0.002960002083841467 % done
On line10000, 0.005920004167682934 % done
On line15000, 0.008880006251524401 % done
On line20000, 0.011840008335365868 % done
On line25000, 0.014800010419207334 % done
On line30000, 0.017760012503048803 % done
On line35000, 0.02072001458689027 % done
On line40000, 0.023680016670731736 % done
On line45000, 0.026640018754573204 % done
On line50000, 0.02960002083841467 % done
On line55000, 0.03256002292225614 % done
On line60000, 0.035520025006097605 % done
On line65000, 0.03848002708993907 % done
On line70000, 0.04144002917378054 % done
On line75000, 0.04440003125762201 % done
On line80000, 0.04736003334146347 % done
On line85000, 0.050320035425304936 % done
On line90000, 0.05328003750914641 % done
On line95000, 0.05624003959298787 % done
On line100000, 0.05920004167682934 % done
On line105000, 0.06216004376067081 % done
On line110000, 0.06512004584451228 % done
On line115000, 0.06808004792835375 % done
On line120000, 0.071040

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   29.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   29.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Pa

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              rand

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = GS_object.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Binary", "Random Forest", "Elec", "Elec-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = GS_object.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Binary", "Random Forest", "Elec", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = GS_object.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["Binary", "Random Forest", "Elec", "ALL-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.0s finished


89.04%


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


84.81%


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s


72.62%


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Binary,Random Forest,Elec,Elec-Train,0.890443,0.890797,0.890603,0.895395
0,Binary,Random Forest,Elec,Elec-Test,0.848115,0.855100,0.850690,0.845975
0,Binary,Random Forest,Elec,ALL-Test,0.726240,0.878691,0.801079,0.690638


In [ ]:
total_results.to_csv('/content/drive/MyDrive/cis519_final/Baselines/RandomForest/total_results.csv')

In [ ]:
total_results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Random Forest,All,All-Train,0.658685,0.661813,0.661848,0.708839
0,5Cat,Random Forest,All,All-Test,0.428357,0.623331,0.484019,0.406110
0,5Cat,Random Forest,All,Elec-Test,0.425952,0.470233,0.454368,0.444021
0,5Cat,Random Forest,Elec,Elec-Train,0.694476,0.695094,0.695243,0.753679
0,5Cat,Random Forest,Elec,Elec-Test,0.469977,0.502783,0.487694,0.489218
0,5Cat,Random Forest,Elec,ALL-Test,0.386031,0.622710,0.425438,0.372395
0,Binary,Random Forest,All,All-Train,0.876949,0.877206,0.877183,0.880323
0,Binary,Random Forest,All,All-Test,0.743919,0.879429,0.848142,0.701758
0,Binary,Random Forest,All,Elec-Test,0.823400,0.827400,0.836942,0.820836
0,Binary,Random Forest,Elec,Elec-Train,0.890443,0.890797,0.890603,0.895395


All Neutrality

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/RandomForest/results_All_neut.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_neutrality_binary.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_neutrality_binary_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_neutrality_binary_test.csv')

depth = [int(x) for x in np.linspace(10, 40, num = 5)]
# estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 5)]
grid = {'max_depth': depth}

rf = RandomForestClassifier(verbose = 1, n_jobs=-1)

GS_object = GridSearchCV(estimator = rf, param_grid = grid, verbose = 1)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['neutrality']
y_test = df_test['neutrality']
y_exp = df_exp['neutrality']


GS_object.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   12.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   11.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Pa

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              rand

In [ ]:
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = GS_object.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Neutrality", "Random Forest", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = GS_object.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Neutrality", "Random Forest", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores


results.to_csv(output_dir, index=False)
total_results = total_results.append(results)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    1.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    2.3s finished


83.04%


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.9s finished


59.67%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Neutrality,Random Forest,All,All-Train,0.830366,0.831156,0.831151,0.837432
0,Neutrality,Random Forest,All,All-Test,0.596668,0.746769,0.724450,0.596113
